# Oscillator frequency error estimator simulation

Implementation of a Kalman filter for estimating timer frequency and phase error against a reference clock.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.seterr(over='ignore');

Load CSV containing parts of corresponding local and remote timestamps

In [ ]:
data = np.loadtxt("sync_temp2.csv", delimiter=',', dtype=np.uint32)
time_wrap = np.iinfo(data.dtype).max + 1

Function to transform data as if it was generated by a clock at a different frequency.

In [ ]:
def clock_divide(data, divider):
    return ((np.unwrap(data.astype(np.float64), period=time_wrap, axis=0) / divider) % time_wrap).astype(np.uint32)

Simulate the frequency estimator using an implementation that is as similar as possible to the C version in the firmware. All the matrix operations have been expanded and simplified using sympy (see `freq_est_model.ipynb`)

In [ ]:
def freq_est(data, nominal_freq, q_theta, q_f, r, p0):
    nominal_freq_2 = (nominal_freq * nominal_freq).astype(np.float32)
    q_f = (q_f / nominal_freq_2).astype(np.float32)
    r = (r * nominal_freq_2).astype(np.float32)
    
    last_time = np.uint32(0)

    t = np.empty(data.shape[0], dtype=np.uint64)
    z = np.empty(data.shape[0], dtype=np.float32)
    theta_whole = np.zeros(data.shape[0], dtype=np.uint32)
    theta = np.empty(data.shape[0], dtype=np.float32)
    f = np.empty(data.shape[0], dtype=np.float32)
    P = np.eye(2, dtype=np.float32) * p0
    
    time_wrap = np.iinfo(data.dtype).max + 1
    
    for i, (local_time, central_time) in enumerate(data):    
        if i == 0:
            t[i] = 0
            z[i] = 0
            theta_whole[i] = (central_time - local_time).astype(np.uint32)
            theta[i] = 0.0
            f[i] = 0.0
            last_time = local_time
            continue

        z_unsigned = (central_time - (local_time + theta_whole[i - 1])).astype(np.uint32)

        if z_unsigned > (time_wrap / 2):
            z[i] = -(time_wrap - z_unsigned).astype(np.float32)
        else:
            z[i] = z_unsigned.astype(np.float32)
        
        dt = (local_time - last_time).astype(np.uint32).astype(np.float32)
        last_time = local_time
        t[i] = t[i - 1] + dt
    
        theta[i] = theta[i - 1] + dt * f[i - 1]
        f[i] = f[i - 1]
    
        dt_p11 = dt * P[1, 1];
        P[0, 0] += dt * (dt * q_theta + P[0, 1] + P[1, 0] + dt_p11)
        P[0, 1] += dt_p11
        P[1, 0] += dt_p11
        P[1, 1] += dt * dt * q_f
    
        p00_r = P[0, 0] + r
        k0 = P[0, 0] / p00_r
        k1 = P[1, 0] / p00_r
    
        theta_error = z[i] - theta[i]
        theta[i] += k0 * theta_error
        f[i] += k1 * theta_error

        delta_theta_whole = theta[i].astype(np.int32)
        theta_whole[i] = theta_whole[i - 1] + delta_theta_whole
        theta[i] -= delta_theta_whole
    
        P[1, 1] -= P[0, 1] * P[1, 0] / p00_r
        P[0, 1] = r * P[0, 1] / p00_r
        P[0, 0] = r * k0
        P[1, 0] = r * k1

    return t, theta_whole + z, theta_whole + theta, f

Simulate and plot the filter at the raw frequency and simulated higher frequency. This verifies that the parameter scaling is working as expected.

In [ ]:
nominal_freq=np.float32(16e6)
q_theta=np.float32(0)
q_f=np.float32(256)
r=np.float32(390625)
p0=np.float32(1e6)
t, z, theta, f = freq_est(data, nominal_freq=nominal_freq, q_theta=q_theta, q_f=q_f, r=r, p0=p0)
t_sec = t / nominal_freq

divider = 0.1
nominal_freq /= divider
t_d, z_d, theta_d, f_d = freq_est(clock_divide(data, divider), nominal_freq=nominal_freq, q_theta=q_theta, q_f=q_f, r=r, p0=p0)
t_d_sec = t_d / nominal_freq

fig, (ax_freq, ax_theta) = plt.subplots(2, 1, figsize=(6,8), sharex=True)
ax_freq.plot(t_sec, f, t_d_sec, f_d)
ax_freq.set_title("Frequency")
ax_freq.set_ylabel("f")
ax_freq.legend(("Filtered", "Filtered (divided)"));

ax_theta.plot(t_sec, theta - theta[0], t_d_sec, (theta_d - theta_d[0]) * divider, t_sec, z.astype(np.float64) - z[0]);
ax_theta.set_title("Phase")
ax_theta.set_ylabel("θ")
ax_theta.set_xlabel("t");
ax_theta.legend(("Filtered", "Filtered (divided)", "Measured"));

Alternative implementation using matrix operations.

In [ ]:
def freq_est_matrix(data, nominal_freq, q_theta, q_f, r, p0):
    nominal_freq_2 = (nominal_freq * nominal_freq).astype(np.float32)
    q_f = (q_f / nominal_freq_2).astype(np.float32)
    r = (r * nominal_freq_2).astype(np.float32)

    Q = np.diag([q_theta, q_f])
    R = np.array([[r]])

    I = np.eye(2)
    C = np.array([[1, 0]])
    
    last_time = np.uint32(0)
    offset = np.uint32(0)

    t = np.empty(data.shape[0], dtype=np.uint64)
    z = np.empty(data.shape[0], dtype=np.float32)
    x = np.empty((2, data.shape[0]))
    P = np.eye(2, dtype=np.float32) * p0

    time_wrap = np.iinfo(data.dtype).max + 1

    for i, (local_time, central_time) in enumerate(data):    
        if i == 0:
            t[i] = 0
            z[i] = 0
            offset = (central_time - local_time).astype(np.uint32)
            x[0, i] = 0.0
            x[1, i] = 0.0
            last_time = local_time
            continue

        z_unsigned = (central_time - (local_time + offset)).astype(np.uint32)

        if z_unsigned > (time_wrap / 2):
            z[i] = -(time_wrap - z_unsigned).astype(np.float32)
        else:
            z[i] = z_unsigned.astype(np.float32)
        
        dt = (local_time - last_time).astype(np.uint32).astype(np.float32)
        last_time = local_time
        t[i] = t[i - 1] + dt

        # Predict
        A = np.array([[1, dt], [0, 1]])
        x[:, i] = A @ x[:, i - 1]
        P = A @ P @ A.T + Q * dt**2

        # Update
        K = P @ C.T / (C @ P @ C.T + R)
        x[:, i] += K @ (z[i] - C @ x[:, i])
        P = (I - K @ C) @ P @ (I - K @ C).T + K @ R @ K.T
    
    return t, z, x[0, :], x[1, :]

In [ ]:
nominal_freq=np.float32(16e6)
q_theta=np.float32(0)
q_f=np.float32(256)
r=np.float32(390625)
p0=np.float32(1e6)
t, z, theta, f = freq_est_matrix(data, nominal_freq=nominal_freq, q_theta=q_theta, q_f=q_f, r=r, p0=p0)
t_sec = t / nominal_freq

fig, (ax_freq, ax_theta) = plt.subplots(2, 1, figsize=(6,8), sharex=True)
ax_freq.plot(t_sec, f)
ax_freq.set_title("Frequency")
ax_freq.set_ylabel("f")

ax_theta.plot(t_sec, theta - theta[0], t_sec, z.astype(np.float64) - z[0]);
ax_theta.set_title("Phase")
ax_theta.set_ylabel("θ")
ax_theta.set_xlabel("t");
ax_theta.legend(("Filtered", "Measured"));